# Importing libraries, Examining data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [2]:
# Uploading csv
originaldf = pd.read_csv('https://raw.githubusercontent.com/Bomsk/DA3_2023/main/Assignment%201/morg-2014-emp.csv')

In [3]:
originaldf.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


In [4]:
# Filtering for Education, Training, and Library Occupations
df = originaldf[
    (originaldf['occ2012'] >= 2200) &    
    (originaldf['occ2012'] <= 2550)]

df.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
7,15,199270459680970,January,AL,3633.3439,900.00,39,44,2,NaN,...,3,13,"Native, Born In US",63,"Colleges and universities, including junior co...",2200,"Private, For Profit",No,No,Employed-At Work
18,36,875095569100620,January,AL,3572.2984,102.00,12,39,2,NaN,...,0,0,"Native, Born In US",63,Elementary and secondary schools (6111),2340,Government - Local,No,No,Employed-At Work
19,37,875095569100620,January,AL,3572.2984,102.00,12,39,2,NaN,...,0,0,"Native, Born In US",63,Elementary and secondary schools (6111),2340,Government - Local,No,No,Employed-At Work
21,43,954001919079770,January,AL,2847.5801,826.92,40,44,1,NaN,...,0,0,"Native, Born In US",63,Elementary and secondary schools (6111),2310,"Private, For Profit",No,No,Employed-At Work
30,55,981709270829007,January,AL,3102.3708,68.00,8,40,1,NaN,...,0,0,"Native, Born In US",63,Elementary and secondary schools (6111),2310,"Private, For Profit",No,No,Employed-At Work


In [5]:
# Adding gender, wage, lnwage, and agesq features.
df["female"] = (df["sex"] == 2)
df["w"] = df["earnwke"] / df["uhours"]
df["lnw"] = np.log(df["w"])
df["agesq"] = np.power(df["age"], 2)

df["grade92"] = (df["grade92"] >= 43)                       # Education: Bachelor to Doctorate
df["race"] = (df["race"] == 1)                              # Race: White

def has_gov(text):                                          # Government: Federal, State, or Local
    return 'Government' in text

df["government"] = df["class"].apply(has_gov)

def has_native(text):                                       # Native: born in US, Puerto Rico, or US Outlying Area, or Abroad of American Parent(s)
    return 'Native' in text

df["native"] = df["prcitshp"].apply(has_native)



# Building Models

In [6]:
# OLS regression
reg1 = smf.ols(formula="w~age", data=df).fit(cov_type = "HC1")
reg2 = smf.ols(formula="w~age+sex", data=df).fit(cov_type = "HC1")
reg3 = smf.ols(formula="w~age+sex+grade92+race", data=df).fit(cov_type = "HC1")
reg4 = smf.ols(formula="w~age+sex+grade92+race+government+native", data=df).fit(cov_type = "HC1")

In [7]:
# Calculating BIC 
bic = [round(x.bic, 2) for x in [reg1,reg2,reg3,reg4]]

# Creating summary table
sg = stargazer.Stargazer([reg1,reg2,reg3,reg4])
sg.add_line('BIC', bic, location=stargazer.LineLocation.FOOTER_BOTTOM)

# Customizing covariate order
sg.covariate_order(["Intercept", "age", "sex", "race[T.True]", "grade92[T.True]", "native[T.True]", "government[T.True]"])

# Renaming covariates 
cov_renamed = {
    'Intercept': 'Constant', 
    'age': 'Age', 
    'sex': 'Sex', 
    'race[T.True]': 'Race: White', 
    'grade92[T.True]': 'Education: Bachelor to Doctorate', 
    'native[T.True]': 'Native: born in US, Puerto Rico, or US Outlying Area, or Abroad of American Parent(s)',
    'government[T.True]': 'Government: Federal, State, or Local' 
    
}
sg.rename_covariates(cov_renamed)

sg

In [8]:
# Calculating RMSE 
rmse1 = rmse(reg1.fittedvalues,df.w)
rmse2 = rmse(reg2.fittedvalues,df.w)
rmse3 = rmse(reg3.fittedvalues,df.w)
rmse4 = rmse(reg4.fittedvalues,df.w)

print(rmse1)
print(rmse2)
print(rmse3)
print(rmse4)

19.48376709945986
19.348262133676347
18.960287319162695
18.958534099191958


In [9]:
# Cross-validating OLS regression
from sklearn.model_selection import KFold
k = KFold(n_splits=4, shuffle=False, random_state=None)

### Cross validate OLS with combining sklearn k-fold cross validation and statsmodels ols formula


def cv_reg(formula, data, kfold, robustse=None):
    regression_list = []
    predicts_on_test = []
    rsquared = []
    rmse_list = []

    # Calculating OLS for each fold

    for train_index, test_index in k.split(data):
        # print("TRAIN:", train_index, "TEST:", test_index)
        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]
        if robustse is None:
            model = smf.ols(formula, data=data_train).fit()
        else:
            model = smf.ols(formula, data=data_train).fit(cov_type=robustse)
        regression_list += [model]
        predicts_on_test += [model.predict(data_test)]
        rsquared += [model.rsquared]
        rmse_list += [rmse(data_train[formula.split("~")[0]], model.predict())]

    return {
        "regressions": regression_list,
        "test_predict": predicts_on_test,
        "r2": rsquared,
        "rmse": rmse_list,
    }


def summarize_cv(cvlist, stat="rmse"):
    result = pd.DataFrame(
        {"Model" + str(x + 1): cvlist[x][stat] for x in range(len(cv_list))}
    )
    result["Resample"] = ["Fold" + str(x + 1) for x in range(len(cvlist[0]["rmse"]))]
    result = result.set_index("Resample")
    result = pd.concat([result, pd.DataFrame(result.mean(), columns=["Average"]).T])
    return result



In [10]:
cv1 = cv_reg("w~age", df, k, "HC1")
cv2 = cv_reg("w~age+sex", df, k, "HC1")
cv3 = cv_reg("w~age+sex+grade92+race", df, k, "HC1")
cv4 = cv_reg("w~age+sex+grade92+race+government+native", df, k,"HC1")
cv_list = [cv1, cv2, cv3, cv4]



In [11]:
summarize_cv(cv_list)

,Model1,Model2,Model3,Model4
Fold1,21.177536,21.054421,20.683905,20.683405
Fold2,19.646234,19.511919,19.116726,19.113852
Fold3,15.904923,15.717255,15.264806,15.261016
Fold4,20.755919,20.642724,20.282369,20.280114
Average,19.371153,19.231580,18.836951,18.834597
